In [1]:
import os
import pandas as pd
import numpy as np
from scipy.integrate import trapz
from scipy.stats import skew, kurtosis, iqr

# Set TTM
ttm = 27

# Create directory for output if it doesn't exist
output_dir = "RiskPremia/Tau-independent/unique/moneyness_step_0d01/multivariate_clustering_9_27_45/Bitcoin_Premium/"
os.makedirs(output_dir, exist_ok=True)

# Load sample returns
RR_OA = pd.read_excel(os.path.join(output_dir, "477_sample_return_OA.xlsx"))
RR_c0 = pd.read_excel(os.path.join(output_dir, "477_sample_return_HV.xlsx"))
RR_c1 = pd.read_excel(os.path.join(output_dir, "477_sample_return_LV.xlsx"))

# Compute annualized moments
P_mu_OA_back = RR_OA["return_t_minus_27"].mean() * 365 / ttm
P_sigma_OA_back = RR_OA["simpleRV"].mean() * 365 / ttm

P_mu_c0_back = RR_c0["return_t_minus_27"].mean() * 365 / ttm
P_sigma_c0_back = RR_c0["simpleRV"].mean() * 365 / ttm

P_mu_c1_back = RR_c1["return_t_minus_27"].mean() * 365 / ttm
P_sigma_c1_back = RR_c1["simpleRV"].mean() * 365 / ttm

P_mu_OA_for = RR_OA["return_t_plus_27"].mean() * 365 / ttm
P_sigma_OA_for = RR_OA["simpleFV"].mean() * 365 / ttm

P_mu_c0_for = RR_c0["return_t_plus_27"].mean() * 365 / ttm
P_sigma_c0_for = RR_c0["simpleFV"].mean() * 365 / ttm

P_mu_c1_for = RR_c1["return_t_plus_27"].mean() * 365 / ttm
P_sigma_c1_for = RR_c1["simpleFV"].mean() * 365 / ttm

# Report summary
Moments_summary = pd.DataFrame([
    [P_mu_OA_back, P_mu_c0_back, P_mu_c1_back, P_mu_OA_for, P_mu_c0_for, P_mu_c1_for],
    [P_sigma_OA_back, P_sigma_c0_back, P_sigma_c1_back, P_sigma_OA_for, P_sigma_c0_for, P_sigma_c1_for]
], index=["Ann Mean", "Ann Variance"],
   columns=["Overall", "Cluster 0", "Cluster 1", "Overall", "Cluster 0", "Cluster 1"])

print("P moments backward, P moments forward")
print(Moments_summary)

# Functions for moments calculations
def density_moments(ret, density, ttm):
    moments = np.zeros(4)
    moments[0] = trapz(density * ret, ret)  # 1st moment
    moments[1] = trapz(density * (ret - moments[0])**2, ret)  # 2nd moment
    moments[2] = trapz(density * (ret - moments[0])**3, ret)  # 3rd moment
    moments[3] = trapz(density * (ret - moments[0])**4, ret)  # 4th moment

    mean = moments[0] * 365 / ttm
    variance = moments[1] * 365 / ttm
    skewness = moments[2] / moments[1]**1.5
    kurtosis_val = moments[3] / moments[1]**2

    return np.array([mean, variance, skewness, kurtosis_val])

def sample_moments(returns, ttm):
    moments = np.zeros(4)
    moments[0] = np.mean(returns)
    moments[1] = np.var(returns)
    moments[2] = skew(returns)
    moments[3] = kurtosis(returns)

    mean = moments[0] * 365 / ttm
    variance = moments[1] * 365 / ttm
    skewness = moments[2]
    kurtosis_val = moments[3]

    return np.array([mean, variance, skewness, kurtosis_val])

def sample_moments_iqr(returns, ttm):
    moments = np.zeros(4)
    moments[0] = np.mean(returns)
    moments[1] = iqr(returns)**2  # Approximation of variance
    moments[2] = np.mean((returns - moments[0])**3)
    moments[3] = np.mean((returns - moments[0])**4)

    mean = moments[0] * 365 / ttm
    variance = moments[1] * 365 / ttm
    skewness = moments[2] / moments[1]**1.5
    kurtosis_val = moments[3] / moments[1]**2

    return np.array([mean, variance, skewness, kurtosis_val])


P moments backward, P moments forward
              Overall  Cluster 0  Cluster 1   Overall  Cluster 0  Cluster 1
Ann Mean      0.18006   0.050212   0.404796  0.598557   0.443041   0.867719
Ann Variance  2.55124   2.919922   1.913135  2.514835   2.623076   2.327495
